In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SQLContext, SparkConf

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
restaurant_schema = StructType([
    StructField('Restaurant_Name', StringType(), True),
    StructField('City', StringType(), True),
    StructField('Restaurant_ID', IntegerType(), False),
    StructField('Has_Table_booking', StringType(), True),
    StructField('Has_Online_delivery', StringType(), True),
    StructField('Is_delivering_now', StringType(), True),
    StructField('Country_Code', IntegerType(), False)
            ])

foods_schema = StructType([
    StructField('Country_Code', IntegerType(), False),
    StructField('Cuisine', StringType(), False),
    StructField('Rating', FloatType(), False),
    StructField('Rating_text', StringType(), False),
    StructField('Votes', IntegerType(), False)
    
                          ])


cost_schema = StructType([
    StructField('Cost_per_person', IntegerType(), False),
    StructField('Currency', StringType(), True),
    StructField('Price_range', IntegerType(), False),
    StructField('Country_Code', IntegerType(), False)
                          ])

country_schema = StructType([
    StructField('Country_Code', IntegerType(), False),
    StructField('Country', StringType(), True),
    StructField('Per_Capita_Income(USD)', IntegerType(), False)
                ])

cost_new_schema = StructType([
    StructField('Cost_per_person', IntegerType(), False),
    StructField('Currency', StringType(), True),
    StructField('Price_range', IntegerType(), False),
    StructField('Country_Code', IntegerType(), False),
    StructField('Cost_USD', FloatType(), False)
                    ])



In [6]:
Restaurant=spark.read.csv("hdfs://localhost:9000/fuse_project/Zomato_project/Restaurant.csv", header=True, schema = restaurant_schema )
Foods=spark.read.csv("hdfs://localhost:9000/fuse_project/Zomato_project/Foods.csv", header=True, schema = foods_schema)
Cost=spark.read.csv("hdfs://localhost:9000/fuse_project/Zomato_project/Cost.csv", header=True, schema = cost_schema)
Country=spark.read.csv("hdfs://localhost:9000/fuse_project/Zomato_project/Country.csv" , header=True, schema = country_schema)
Cost_new=spark.read.csv("hdfs://localhost:9000/fuse_project/Zomato_project/Cost_new.csv" , header=True, schema = cost_new_schema)

In [7]:
Restaurant.printSchema()
Foods.printSchema()
Cost.printSchema()
Country.printSchema()
Cost_new.printSchema()


root
 |-- Restaurant_Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Restaurant_ID: integer (nullable = true)
 |-- Has_Table_booking: string (nullable = true)
 |-- Has_Online_delivery: string (nullable = true)
 |-- Is_delivering_now: string (nullable = true)
 |-- Country_Code: integer (nullable = true)

root
 |-- Country_Code: integer (nullable = true)
 |-- Cuisine: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Rating_text: string (nullable = true)
 |-- Votes: integer (nullable = true)

root
 |-- Cost_per_person: integer (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Price_range: integer (nullable = true)
 |-- Country_Code: integer (nullable = true)

root
 |-- Country_Code: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Per_Capita_Income(USD): integer (nullable = true)

root
 |-- Cost_per_person: integer (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Price_range: integer (nullable = true

In [27]:
Restaurant.show(5, truncate=False)
Foods.show(5)
Cost.show(5)
Country.show()
Cost_new.show()

+----------------------+----------------+-------------+-----------------+-------------------+-----------------+------------+
|Restaurant_Name       |City            |Restaurant_ID|Has_Table_booking|Has_Online_delivery|Is_delivering_now|Country_Code|
+----------------------+----------------+-------------+-----------------+-------------------+-----------------+------------+
|Le Petit Souffle      |Makati City     |6317637      |Yes              |No                 |No               |162         |
|Izakaya Kikufuji      |Makati City     |6304287      |Yes              |No                 |No               |162         |
|Heat - Edsa Shangri-La|Mandaluyong City|6300002      |Yes              |No                 |No               |162         |
|Ooma                  |Mandaluyong City|6318506      |No               |No                 |No               |162         |
|Sambo Kojin           |Mandaluyong City|6314302      |Yes              |No                 |No               |162         |



# 1. Cities with maximum Restaurants

In [12]:

task1= Restaurant.groupBy("City").count().sort(col("count").desc())



In [13]:
task1.show()

+------------+-----+
|        City|count|
+------------+-----+
|   New Delhi| 5473|
|     Gurgaon| 1118|
|       Noida| 1080|
|   Faridabad|  251|
|   Ghaziabad|   25|
|    Amritsar|   21|
|   Ahmedabad|   21|
|Bhubaneshwar|   21|
|     Lucknow|   21|
|    Guwahati|   21|
|  Manchester|   20|
|    Vadodara|   20|
|     Colombo|   20|
|   Tampa Bay|   20|
|   Bangalore|   20|
|    Valdosta|   20|
|     Chennai|   20|
|       Kochi|   20|
|    Savannah|   20|
|    Auckland|   20|
+------------+-----+
only showing top 20 rows



# 2. Which cuisine is famous in Ahmedabad city?

In [14]:
task2= Restaurant.join(Foods, Restaurant.Country_Code== Foods.Country_Code,"inner").select("City","Cuisine","Rating_text")

In [15]:
task21=task2.filter(task2.City=="Ahmedabad")

In [17]:
task21.show(10,truncate=False)

+---------+------------------------+-----------+
|City     |Cuisine                 |Rating_text|
+---------+------------------------+-----------+
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
|Ahmedabad|[North Indian,  Mughlai]|Good       |
+---------+------------------------+-----------+
only showing top 10 rows



In [22]:
task22 = task21.select(task21.City,explode(task21.Cuisine)).withColumnRenamed("col","Cuisine")
#task22.show(10)

In [31]:
task23=task22.groupBy("Cuisine").count().distinct().sort(col("count").desc()).show()


+-------------+-----+
|      Cuisine|count|
+-------------+-----+
| North Indian|62727|
|      Chinese|39039|
|    Fast Food|27384|
| North Indian|20076|
|      Chinese|17388|
|      Mughlai|16275|
|    Fast Food|13818|
|       Bakery|12936|
|         Cafe|11781|
|      Italian|10605|
|  Continental|10395|
|     Desserts| 9786|
| South Indian| 7749|
|  Street Food| 6720|
| South Indian| 5481|
|       Mithai| 5166|
|  Street Food| 4914|
|  Continental| 4809|
|      Mughlai| 4515|
|        Pizza| 4389|
+-------------+-----+
only showing top 20 rows



In [32]:
type(task22)

pyspark.sql.dataframe.DataFrame

OR

In [23]:
task_22=task21.filter(task21.Rating_text == "Excellent")
task_22.show()

+---------+--------------------+-----------+
|     City|             Cuisine|Rating_text|
+---------+--------------------+-----------+
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad|[Cafe,  North Ind...|  Excellent|
|Ahmedabad

In [24]:
task_23 = task_22.select(task_22.Rating_text,explode(task_22.Cuisine)).withColumnRenamed("col","Cuisine")
task_23.groupBy("Cuisine").count().sort(col("count").desc()).show()

+--------------+-----+
|       Cuisine|count|
+--------------+-----+
|  North Indian|  567|
|   Continental|  378|
|       Italian|  357|
|  North Indian|  336|
|       Chinese|  336|
|      American|  273|
|          Cafe|  252|
| Mediterranean|  210|
|   Continental|  210|
|          Cafe|  210|
|         Asian|  189|
|     Fast Food|  168|
|      European|  168|
|       Italian|  147|
|        Burger|  147|
|     Ice Cream|  126|
|       Mughlai|  126|
|       Mexican|  105|
|      Desserts|  105|
|       Mexican|  105|
+--------------+-----+
only showing top 20 rows



# 3.how per capita income affects food ordering behaviour?

In [25]:
task3= Foods.join(Country,"Country_Code","inner")
task3.show()

+------------+------+-----------+-----+--------------------+-----------+----------------------+
|Country_Code|Rating|Rating_text|Votes|             Cuisine|    Country|Per_Capita_Income(USD)|
+------------+------+-----------+-----+--------------------+-----------+----------------------+
|         162|   4.8|  Excellent|  314|[French,  Japanes...|Phillipines|                  3160|
|         162|   4.5|  Excellent|  591|          [Japanese]|Phillipines|                  3160|
|         162|   4.4|  Very Good|  270|[Seafood,  Asian,...|Phillipines|                  3160|
|         162|   4.9|  Excellent|  365|  [Japanese,  Sushi]|Phillipines|                  3160|
|         162|   4.8|  Excellent|  229| [Japanese,  Korean]|Phillipines|                  3160|
|         162|   4.4|  Very Good|  336|           [Chinese]|Phillipines|                  3160|
|         162|   4.0|  Very Good|  520|  [Asian,  European]|Phillipines|                  3160|
|         162|   4.2|  Very Good|  677|[

In [26]:
task31= Country.select(max("Per_Capita_Income(USD)"), min("Per_Capita_Income(USD)")).show()

+---------------------------+---------------------------+
|max(Per_Capita_Income(USD))|min(Per_Capita_Income(USD))|
+---------------------------+---------------------------+
|                      68309|                       2100|
+---------------------------+---------------------------+



In [28]:
task3= task3.withColumnRenamed("Per_Capita_Income(USD)","PCI")

In [30]:
#task3.show()

In [31]:
task33= task3.filter(task3.PCI==68309).select(task3.Country,task3.PCI,task3.Rating,explode(task3.Cuisine)).withColumnRenamed("col","Cuisine").show(10)


+-------------+-----+------+--------------+
|      Country|  PCI|Rating|       Cuisine|
+-------------+-----+------+--------------+
|United States|68309|   3.3|           BBQ|
|United States|68309|   3.3|        Burger|
|United States|68309|   3.3|       Seafood|
|United States|68309|   3.3|      American|
|United States|68309|   3.3|           BBQ|
|United States|68309|   3.4|       Mexican|
|United States|68309|   3.4|Coffee and Tea|
|United States|68309|   3.4|      Sandwich|
|United States|68309|   3.5|     Fast Food|
|United States|68309|   3.9|         Asian|
+-------------+-----+------+--------------+
only showing top 10 rows



# 4. Resturants with maximum ratings?

In [32]:
task4= Restaurant.join(Foods, "Country_Code","inner").select("Restaurant_Name","Rating", "Rating_text")
task4.show()

+--------------------+------+-----------+
|     Restaurant_Name|Rating|Rating_text|
+--------------------+------+-----------+
|The Food Hall by ...|   4.8|  Excellent|
|      NIU by Vikings|   4.8|  Excellent|
|Wildflour Cafe + ...|   4.8|  Excellent|
|Hobing Korean Des...|   4.8|  Excellent|
|          Balay Dako|   4.8|  Excellent|
|Nonna's Pasta & P...|   4.8|  Excellent|
|       Cafe Arabelle|   4.8|  Excellent|
|Sodam Korean Rest...|   4.8|  Excellent|
|          Guevarra's|   4.8|  Excellent|
|    Silantro Fil-Mex|   4.8|  Excellent|
|Mad Mark's Creame...|   4.8|  Excellent|
|    Silantro Fil-Mex|   4.8|  Excellent|
|            Locavore|   4.8|  Excellent|
|Spiral - Sofitel ...|   4.8|  Excellent|
|             Vikings|   4.8|  Excellent|
|          Buffet 101|   4.8|  Excellent|
|        Din Tai Fung|   4.8|  Excellent|
|         Sambo Kojin|   4.8|  Excellent|
|                Ooma|   4.8|  Excellent|
|Heat - Edsa Shang...|   4.8|  Excellent|
+--------------------+------+-----

In [33]:
task41=task4.filter(task4.Rating_text=="Excellent").distinct()

In [34]:
print(task41.count())
task41.show()

36834
+--------------------+------+-----------+
|     Restaurant_Name|Rating|Rating_text|
+--------------------+------+-----------+
| Confeitaria Colombo|   4.6|  Excellent|
|          BlackStone|   4.5|  Excellent|
|      Lulu's Waikiki|   4.6|  Excellent|
|HI Lite Bar & Lounge|   4.6|  Excellent|
|Samurai Japanese ...|   4.6|  Excellent|
|Goldy's Breakfast...|   4.6|  Excellent|
|Theo Yianni's Aut...|   4.7|  Excellent|
| The Olde Pink House|   4.7|  Excellent|
|         Los Aztecas|   4.7|  Excellent|
|        Jethro's BBQ|   4.7|  Excellent|
|Chandlers Steakhouse|   4.7|  Excellent|
|     Bait El Khetyar|   4.6|  Excellent|
|        Baati Chokha|   4.9|  Excellent|
|          Wow Noodle|   4.9|  Excellent|
|Earthen Oven - Fo...|   4.9|  Excellent|
|          Cafetorium|   4.9|  Excellent|
|        366 Junction|   4.9|  Excellent|
|        Chatori Gali|   4.9|  Excellent|
|Ram Ram Ji Kachor...|   4.9|  Excellent|
|          Dosa Plaza|   4.9|  Excellent|
+--------------------+------

In [44]:
#task42=task41.select("Restaurant_Name",task41.Rating.cast('float'))

In [35]:
task42=task41.filter(task41.Rating >= 4.9).distinct().show()

+--------------------+------+-----------+
|     Restaurant_Name|Rating|Rating_text|
+--------------------+------+-----------+
|        Baati Chokha|   4.9|  Excellent|
|          Wow Noodle|   4.9|  Excellent|
|Earthen Oven - Fo...|   4.9|  Excellent|
|          Cafetorium|   4.9|  Excellent|
|        366 Junction|   4.9|  Excellent|
|        Chatori Gali|   4.9|  Excellent|
|Ram Ram Ji Kachor...|   4.9|  Excellent|
|          Dosa Plaza|   4.9|  Excellent|
|      Sugary Affairs|   4.9|  Excellent|
|   High Street Cafí©|   4.9|  Excellent|
|WTF - Wraps Toast...|   4.9|  Excellent|
|           Mafia 2.0|   4.9|  Excellent|
|              Barkat|   4.9|  Excellent|
|          Jeet Pizza|   4.9|  Excellent|
|  New Lazeez Tandoor|   4.9|  Excellent|
|        The Bay Leaf|   4.9|  Excellent|
|             Just In|   4.9|  Excellent|
| King Chilly Kitchen|   4.9|  Excellent|
|   Shanu's Food Shop|   4.9|  Excellent|
|       Mahadev Dhaba|   4.9|  Excellent|
+--------------------+------+-----

# 5. How votes affects the price range ?

In [36]:
task5= Foods.join(Cost, "Country_Code","inner")
task5.show()

+------------+------+-----------+-----+--------------------+---------------+--------+-----------+
|Country_Code|Rating|Rating_text|Votes|             Cuisine|Cost_per_person|Currency|Price_range|
+------------+------+-----------+-----+--------------------+---------------+--------+-----------+
|         162|   4.8|  Excellent|  314|[French,  Japanes...|            900|     PHP|          4|
|         162|   4.8|  Excellent|  314|[French,  Japanes...|           1500|     PHP|          4|
|         162|   4.8|  Excellent|  314|[French,  Japanes...|            750|     PHP|          4|
|         162|   4.8|  Excellent|  314|[French,  Japanes...|            300|     PHP|          2|
|         162|   4.8|  Excellent|  314|[French,  Japanes...|            600|     PHP|          3|
|         162|   4.8|  Excellent|  314|[French,  Japanes...|            425|     PHP|          3|
|         162|   4.8|  Excellent|  314|[French,  Japanes...|            400|     PHP|          3|
|         162|   4.8

In [47]:
#Cost=Cost.na.fill(value=0)

In [37]:
Foods.printSchema()

root
 |-- Country_Code: integer (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Rating_text: string (nullable = true)
 |-- Votes: integer (nullable = true)
 |-- Cuisine: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [38]:
task51=task5.select(max("Votes"), min("Votes"))
task51.show()

+----------+----------+
|max(Votes)|min(Votes)|
+----------+----------+
|     10934|         0|
+----------+----------+



In [39]:
task53=task5.filter(task5.Votes==10934).select("Votes","Price_range")

In [43]:
task53=task5.filter(task5.Votes==155).show()

+------------+------+-----------+-----+--------------------+---------------+--------+-----------+
|Country_Code|Rating|Rating_text|Votes|             Cuisine|Cost_per_person|Currency|Price_range|
+------------+------+-----------+-----+--------------------+---------------+--------+-----------+
|           1|   3.4|    Average|  155|[North Indian,  C...|            700|     INR|          3|
|           1|   3.4|    Average|  155|[North Indian,  C...|            750|     INR|          4|
|           1|   3.4|    Average|  155|[North Indian,  C...|            300|     INR|          2|
|           1|   3.4|    Average|  155|[North Indian,  C...|            200|     INR|          2|
|           1|   3.4|    Average|  155|[North Indian,  C...|            300|     INR|          2|
|           1|   3.4|    Average|  155|[North Indian,  C...|            225|     INR|          2|
|           1|   3.4|    Average|  155|[North Indian,  C...|            300|     INR|          2|
|           1|   3.4

In [55]:
task54=task.filter(task.Votes==788).distinct().show()

+-----+------+
|Votes|Rating|
+-----+------+
|  788|   4.1|
+-----+------+



In [56]:
task54=task.filter(task.Votes==10934).distinct().show()

+-----+------+
|Votes|Rating|
+-----+------+
|10934|   4.8|
+-----+------+



# 6.Top 5 popular ratings per counts?

In [40]:
task6=Foods.select("Rating")
#task6=task6.na.fill(0)
task6= task6.filter(task6.Rating<=5.0).groupBy("Rating").count().sort(col("count").desc())
task6.show(5)

+------+-----+
|Rating|count|
+------+-----+
|   0.0| 2148|
|   3.2|  522|
|   3.1|  519|
|   3.4|  498|
|   3.3|  483|
+------+-----+
only showing top 5 rows




# 7. How Table booking and  online delievery increases or decreases food ordering?

In [41]:
task7= Restaurant.join(Foods, "Country_Code", "inner").select("Cuisine", "Has_Table_booking","Has_Online_delivery")
task7.show()

+--------------------+-----------------+-------------------+
|             Cuisine|Has_Table_booking|Has_Online_delivery|
+--------------------+-----------------+-------------------+
|[French,  Japanes...|              Yes|                 No|
|[French,  Japanes...|              Yes|                 No|
|[French,  Japanes...|              Yes|                 No|
|[French,  Japanes...|               No|                 No|
|[French,  Japanes...|              Yes|                 No|
|[French,  Japanes...|               No|                 No|
|[French,  Japanes...|               No|                 No|
|[French,  Japanes...|               No|                 No|
|[French,  Japanes...|              Yes|                 No|
|[French,  Japanes...|               No|                 No|
|[French,  Japanes...|              Yes|                 No|
|[French,  Japanes...|               No|                 No|
|[French,  Japanes...|              Yes|                 No|
|[French,  Japanes...|  

In [43]:
task71= task7.filter(task7.Has_Table_booking == "Yes").groupBy("Has_Table_booking").count()
#task71=task71.select("Has_Table_booking","count").withColumnRenamed("Has_Table_booking", "Has_Table_booking_Yes")
task72= task7.filter(task7.Has_Table_booking == "No").groupBy("Has_Table_booking").count()
#task72=task72.select("Has_Table_booking","count").withColumnRenamed("Has_Table_booking", "Has_Table_booking_No")
task73= task7.filter(task7.Has_Online_delivery == "Yes").groupBy("Has_Online_delivery").count()
task74= task7.filter(task7.Has_Online_delivery == "No").groupBy("Has_Online_delivery").count()


In [44]:
task99= task71.union(task72).show()
task999=task73.union(task74).show()

+-----------------+--------+
|Has_Table_booking|   count|
+-----------------+--------+
|              Yes| 9614860|
|               No|65453273|
+-----------------+--------+

+-------------------+--------+
|Has_Online_delivery|   count|
+-------------------+--------+
|                Yes|20965476|
|                 No|54102657|
+-------------------+--------+



In [40]:
task75=task99.join(task999).show()

+-----------------+--------+-------------------+--------+
|Has_Table_booking|   count|Has_Online_delivery|   count|
+-----------------+--------+-------------------+--------+
|              Yes| 9614860|                Yes|20965476|
|              Yes| 9614860|                 No|54102657|
|               No|65453273|                Yes|20965476|
|               No|65453273|                 No|54102657|
+-----------------+--------+-------------------+--------+



In [33]:
task75=task71.join(task72).show()

+---------------------+-------+--------------------+--------+
|Has_Table_booking_Yes|  count|Has_Table_booking_No|   count|
+---------------------+-------+--------------------+--------+
|                  Yes|9614860|                  No|65453273|
+---------------------+-------+--------------------+--------+



In [59]:
task71= task7.filter(task7.Has_Table_booking == "Yes").groupBy("Has_Table_booking","Cuisine").count().show(50,truncate=False)
task72= task7.filter(task7.Has_Table_booking == "No").groupBy("Has_Table_booking","Cuisine").count().show(50,truncate=False)

+-----------------+-------------------------------------------------------------------+------+
|Has_Table_booking|Cuisine                                                            |count |
+-----------------+-------------------------------------------------------------------+------+
|Yes              |[Chinese,  North Indian,  Italian]                                 |4444  |
|Yes              |[British,  Chinese,  Italian]                                      |1111  |
|Yes              |[North Indian,  Mediterranean,  Continental]                       |2222  |
|Yes              |[Nepalese,  Tibetan]                                               |1111  |
|Yes              |[Japanese,  Chinese]                                               |1111  |
|Yes              |[Cafe,  Continental,  Italian,  Chinese,  North Indian]            |1111  |
|Yes              |[Cafe,  Mexican,  Italian,  North Indian,  Chinese]                |1111  |
|Yes              |[North Indian,  South Indian,  

In [60]:
task73= task7.filter(task7.Has_Online_delivery == "Yes").groupBy("Has_Online_delivery","Cuisine").count().show(50,truncate=False)
task74= task7.filter(task7.Has_Online_delivery == "No").groupBy("Has_Online_delivery","Cuisine").count().show(50,truncate=False)

+-------------------+-------------------------------------------------------------------+------+
|Has_Online_delivery|Cuisine                                                            |count |
+-------------------+-------------------------------------------------------------------+------+
|Yes                |[Chinese,  North Indian,  Italian]                                 |9692  |
|Yes                |[British,  Chinese,  Italian]                                      |2423  |
|Yes                |[North Indian,  Mediterranean,  Continental]                       |4846  |
|Yes                |[Nepalese,  Tibetan]                                               |2423  |
|Yes                |[Japanese,  Chinese]                                               |2423  |
|Yes                |[Cafe,  Continental,  Italian,  Chinese,  North Indian]            |2423  |
|Yes                |[Cafe,  Mexican,  Italian,  North Indian,  Chinese]                |2423  |
|Yes                |[North In

# 8. Which is most liked table booking or online delievery?

In [45]:
task8= Restaurant.select("Has_Table_booking","Has_Online_delivery")

In [46]:
 task8.filter((task8.Has_Table_booking== "Yes") & (task8.Has_Online_delivery== "Yes")).count()

435

In [47]:
task81= task8.filter(task8.Has_Table_booking == "Yes").groupBy("Has_Table_booking").count()
task81.show()

+-----------------+-----+
|Has_Table_booking|count|
+-----------------+-----+
|              Yes| 1158|
+-----------------+-----+



In [48]:
task82= task8.filter(task8.Has_Online_delivery == "Yes").groupBy("Has_Online_delivery").count()
task82.show()

+-------------------+-----+
|Has_Online_delivery|count|
+-------------------+-----+
|                Yes| 2451|
+-------------------+-----+



In [49]:
task83=task81.join(task82).show()

+-----------------+-----+-------------------+-----+
|Has_Table_booking|count|Has_Online_delivery|count|
+-----------------+-----+-------------------+-----+
|              Yes| 1158|                Yes| 2451|
+-----------------+-----+-------------------+-----+



In [67]:
task81.columns

['Has_Table_booking', 'count']

# 9. Display cuisine having price rating 2 and food rating above 4?

In [50]:
task9= Foods.join(Cost, "Country_Code").select("Cuisine","Price_range","Rating")
task9.show(5)

+--------------------+-----------+------+
|             Cuisine|Price_range|Rating|
+--------------------+-----------+------+
|[French,  Japanes...|          4|   4.8|
|[French,  Japanes...|          4|   4.8|
|[French,  Japanes...|          4|   4.8|
|[French,  Japanes...|          2|   4.8|
|[French,  Japanes...|          3|   4.8|
+--------------------+-----------+------+
only showing top 5 rows



In [52]:
task92= task9.select("Rating","Price_range",explode(task9.Cuisine)).withColumnRenamed("col","Cuisine")
task92.show()

+------+-----------+---------+
|Rating|Price_range|  Cuisine|
+------+-----------+---------+
|   4.8|          4|   French|
|   4.8|          4| Japanese|
|   4.8|          4| Desserts|
|   4.8|          4|   French|
|   4.8|          4| Japanese|
|   4.8|          4| Desserts|
|   4.8|          4|   French|
|   4.8|          4| Japanese|
|   4.8|          4| Desserts|
|   4.8|          2|   French|
|   4.8|          2| Japanese|
|   4.8|          2| Desserts|
|   4.8|          3|   French|
|   4.8|          3| Japanese|
|   4.8|          3| Desserts|
|   4.8|          3|   French|
|   4.8|          3| Japanese|
|   4.8|          3| Desserts|
|   4.8|          3|   French|
|   4.8|          3| Japanese|
+------+-----------+---------+
only showing top 20 rows



In [54]:
task93= task92.filter((task92.Price_range==2)& (task92.Rating>4)).show(25, truncate= False)

+------+-----------+----------+
|Rating|Price_range|Cuisine   |
+------+-----------+----------+
|4.8   |2          |French    |
|4.8   |2          | Japanese |
|4.8   |2          | Desserts |
|4.5   |2          |Japanese  |
|4.4   |2          |Seafood   |
|4.4   |2          | Asian    |
|4.4   |2          | Filipino |
|4.4   |2          | Indian   |
|4.9   |2          |Japanese  |
|4.9   |2          | Sushi    |
|4.8   |2          |Japanese  |
|4.8   |2          | Korean   |
|4.4   |2          |Chinese   |
|4.2   |2          |Seafood   |
|4.2   |2          | Filipino |
|4.2   |2          | Asian    |
|4.2   |2          | European |
|4.9   |2          |European  |
|4.9   |2          | Asian    |
|4.9   |2          | Indian   |
|4.8   |2          |Filipino  |
|4.9   |2          |Filipino  |
|4.9   |2          | Mexican  |
|4.2   |2          |American  |
|4.2   |2          | Ice Cream|
+------+-----------+----------+
only showing top 25 rows



# 10. Count Resturants having no table booking and online delievery  but excellent ratings.

In [55]:
task10= Restaurant.join(Foods, "Country_Code")
task10.show(2)

+------------+----------------+-----------+-------------+-----------------+-------------------+-----------------+------+-----------+-----+--------------------+
|Country_Code| Restaurant_Name|       City|Restaurant_ID|Has_Table_booking|Has_Online_delivery|Is_delivering_now|Rating|Rating_text|Votes|             Cuisine|
+------------+----------------+-----------+-------------+-----------------+-------------------+-----------------+------+-----------+-----+--------------------+
|         162|Le Petit Souffle|Makati City|      6317637|              Yes|                 No|               No|   4.5|  Excellent|  618|[American,  Asian...|
|         162|Le Petit Souffle|Makati City|      6317637|              Yes|                 No|               No|   4.7|  Excellent|  535|[Seafood,  Americ...|
+------------+----------------+-----------+-------------+-----------------+-------------------+-----------------+------+-----------+-----+--------------------+
only showing top 2 rows



In [56]:
task11=task10.filter((task10.Has_Table_booking == "No") & (task10.Has_Online_delivery == "No") & (task10.Rating_text == "Excellent"))

In [58]:
task12=task11.select(count("Restaurant_Name")).show()

+----------------------+
|count(Restaurant_Name)|
+----------------------+
|                677551|
+----------------------+



# 11. Top 10 resturants according to their expensiveness / high price.

In [75]:
from currency_converter import CurrencyConverter
cc= CurrencyConverter()

In [76]:
Cost_new=spark.read.csv("/home/deepika/Documents/Zomato_project/Cost_new.csv", header= True, schema= cost_new_schema)
Cost_new.show()

+---------------+--------+-----------+------------+---------+
|Cost_per_person|Currency|Price_range|Country_Code| Cost_USD|
+---------------+--------+-----------+------------+---------+
|            550|     PHP|          3|         162|11.130291|
|            600|     PHP|          3|         162|12.142136|
|           2000|     PHP|          4|         162|40.473785|
|            750|     PHP|          4|         162| 15.17767|
|            750|     PHP|          4|         162| 15.17767|
|            500|     PHP|          3|         162|10.118446|
|           1000|     PHP|          4|         162|20.236893|
|           1000|     PHP|          4|         162|20.236893|
|           3000|     PHP|          4|         162| 60.71068|
|            550|     PHP|          3|         162|11.130291|
|            400|     PHP|          3|         162| 8.094757|
|            450|     PHP|          3|         162| 9.106602|
|            400|     PHP|          3|         162| 8.094757|
|       

In [78]:
task11= Restaurant.join(Cost_new, "Country_Code").select("Restaurant_Name","Cost_USD")
task111=task11.select(max(col("Cost_USD"))).dropDuplicates()
task11.show()

+----------------+---------+
| Restaurant_Name| Cost_USD|
+----------------+---------+
|Le Petit Souffle|18.213203|
|Le Petit Souffle| 30.35534|
|Le Petit Souffle| 15.17767|
|Le Petit Souffle| 6.071068|
|Le Petit Souffle|12.142136|
|Le Petit Souffle| 8.600679|
|Le Petit Souffle| 8.094757|
|Le Petit Souffle|7.0829124|
|Le Petit Souffle|10.118446|
|Le Petit Souffle| 8.094757|
|Le Petit Souffle| 9.106602|
|Le Petit Souffle| 8.094757|
|Le Petit Souffle|11.130291|
|Le Petit Souffle| 60.71068|
|Le Petit Souffle|20.236893|
|Le Petit Souffle|20.236893|
|Le Petit Souffle|10.118446|
|Le Petit Souffle| 15.17767|
|Le Petit Souffle| 15.17767|
|Le Petit Souffle|40.473785|
+----------------+---------+
only showing top 20 rows




# 12. Food rating and cost per person of countries having per capita income below 5000 

In [82]:
task11= Foods.join(Cost_new,"Country_Code").join(Country, "Country_Code")
task11=task11.withColumnRenamed('Per_Capita_Income(USD)', "PCI")
Cost_new.columns

['Cost_per_person', 'Currency', 'Price_range', 'Country_Code', 'Cost_USD']

In [87]:
task111= task11.select("Rating", "Cost_USD",'PCI').filter(task11.PCI<5000).distinct().show()

+------+----------+----+
|Rating|  Cost_USD| PCI|
+------+----------+----+
|   4.0|0.66943735|2100|
|   4.3| 26.777494|2100|
|   3.4| 2.3430307|2100|
|   4.9| 18.744246|2100|
|   4.9| 7.6985297|2100|
|   4.9| 11.045716|2100|
|   4.5| 23.430307|2100|
|   3.1| 5.6902175|2100|
|   3.1|   5.02078|2100|
|   2.7| 14.058185|2100|
|   2.0| 33.471867|2100|
|   1.9|  6.359655|2100|
|   3.9| 5.6735744|4450|
|   4.9| 7.0829124|3160|
|   4.2| 18.213203|3160|
|   3.9|  4.016624|2100|
|   3.4| 24.434464|2100|
|   1.8| 5.6902175|2100|
|   2.0| 43.513428|2100|
|   2.3| 3.6819055|2100|
+------+----------+----+
only showing top 20 rows



# 13. Which location/ city in a country is most profitable (has high orders) ?

In [112]:
task13= Restaurant.join(Country, "Country_Code")
task13.columns

['Country_Code',
 'Restaurant_Name',
 'City',
 'Restaurant_ID',
 'Has_Table_booking',
 'Has_Online_delivery',
 'Is_delivering_now',
 'Country',
 'Per_Capita_Income(USD)']

In [108]:
#task131= task13.select("Restaurant_Name","City","Country").groupBy("Restaurant_Name","City").count().sort(col("count").desc())
#task131.show()

+--------------------+---------+-----+
|     Restaurant_Name|     City|count|
+--------------------+---------+-----+
|     Cafe Coffee Day|New Delhi|   57|
|      Domino's Pizza|New Delhi|   55|
|              Subway|New Delhi|   38|
|    Green Chick Chop|New Delhi|   37|
|          McDonald's|New Delhi|   33|
|           Keventers|New Delhi|   24|
|               Giani|New Delhi|   24|
|             Giani's|New Delhi|   17|
|          Wah Ji Wah|New Delhi|   14|
|     Aggarwal Sweets|New Delhi|   14|
|         Sagar Ratna|New Delhi|   13|
|  Pizza Hut Delivery|New Delhi|   13|
|     Cafe Coffee Day|    Noida|   13|
|      Baskin Robbins|New Delhi|   13|
| Republic of Chicken|New Delhi|   13|
|   Twenty Four Seven|New Delhi|   13|
|           Pizza Hut|New Delhi|   12|
|34, Chowringhee Lane|New Delhi|   12|
|      Domino's Pizza|    Noida|   11|
|              Subway|  Gurgaon|   11|
+--------------------+---------+-----+
only showing top 20 rows



In [114]:
task131= task13.filter(task13.Country=="India")
task131.show(5)

+------------+--------------------+----+-------------+-----------------+-------------------+-----------------+-------+----------------------+
|Country_Code|     Restaurant_Name|City|Restaurant_ID|Has_Table_booking|Has_Online_delivery|Is_delivering_now|Country|Per_Capita_Income(USD)|
+------------+--------------------+----+-------------+-----------------+-------------------+-----------------+-------+----------------------+
|           1|          Jahanpanah|Agra|      3400025|               No|                 No|               No|  India|                  2100|
|           1| Rangrezz Restaurant|Agra|      3400341|               No|                 No|               No|  India|                  2100|
|           1|Time2Eat - Mama C...|Agra|      3400005|               No|                 No|               No|  India|                  2100|
|           1|Chokho Jeeman Mar...|Agra|      3400021|               No|                 No|               No|  India|                  2100|
|     

In [116]:
task132= task131.groupBy("Restaurant_Name","City").count().sort(col("count").desc())

In [117]:
task132.show()

+--------------------+---------+-----+
|     Restaurant_Name|     City|count|
+--------------------+---------+-----+
|     Cafe Coffee Day|New Delhi|   57|
|      Domino's Pizza|New Delhi|   55|
|              Subway|New Delhi|   38|
|    Green Chick Chop|New Delhi|   37|
|          McDonald's|New Delhi|   33|
|           Keventers|New Delhi|   24|
|               Giani|New Delhi|   24|
|             Giani's|New Delhi|   17|
|          Wah Ji Wah|New Delhi|   14|
|     Aggarwal Sweets|New Delhi|   14|
|         Sagar Ratna|New Delhi|   13|
| Republic of Chicken|New Delhi|   13|
|     Cafe Coffee Day|    Noida|   13|
|  Pizza Hut Delivery|New Delhi|   13|
|      Baskin Robbins|New Delhi|   13|
|   Twenty Four Seven|New Delhi|   13|
|           Pizza Hut|New Delhi|   12|
|34, Chowringhee Lane|New Delhi|   12|
|         Bikanervala|New Delhi|   11|
|              Subway|  Gurgaon|   11|
+--------------------+---------+-----+
only showing top 20 rows



In [121]:
task134= task131.filter(task131.Restaurant_Name == "Cafe Coffee Day").groupBy("City").count()
task134.show()

+---------+-----+
|     City|count|
+---------+-----+
|Faridabad|    1|
|    Noida|   13|
|  Gurgaon|   11|
|Ghaziabad|    1|
|New Delhi|   57|
+---------+-----+



# 14. Restaurants having most number of branches.

In [122]:
task14= Restaurant.select("Restaurant_Name","City").distinct()
task14.show()

+--------------------+---------+
|     Restaurant_Name|     City|
+--------------------+---------+
|       5 Little Pigs|Huskisson|
|               Gupha|Faridabad|
|        Kebab Xpress|Ghaziabad|
|    Fisherman's Cove|      Goa|
|     Mahek By Greenz|  Gurgaon|
|            Fruitpro|  Gurgaon|
|          Apna Dabba|  Gurgaon|
|Special O-cake-sions|  Gurgaon|
|Hide Out The Stre...|   Nagpur|
|           Banzara's|New Delhi|
|      Chicken Vicken|New Delhi|
|Game of Legends -...|New Delhi|
|       The Night Owl|New Delhi|
|          Al- Sheikh|New Delhi|
|Singh Mutton & Ch...|New Delhi|
|         Sona Bakers|New Delhi|
|Shankar Chinese F...|New Delhi|
|Uncultured Cafe &...|New Delhi|
|Chicken Chilli Co...|New Delhi|
|     Rai Ji Caterers|New Delhi|
+--------------------+---------+
only showing top 20 rows



In [123]:
task141= task14.groupBy("Restaurant_Name").count().sort(col("count").desc())
task141.show()

+-----------------+-----+
|  Restaurant_Name|count|
+-----------------+-----+
|  Barbeque Nation|   22|
|        Pizza Hut|   12|
|              KFC|    8|
|The Yellow Chilli|    7|
|            Mocha|    7|
|       McDonald's|    7|
|    Pind Balluchi|    7|
|   Domino's Pizza|    7|
|   Mainland China|    6|
|     TGI Friday's|    6|
|       Farzi Cafe|    6|
|      Sagar Ratna|    5|
|   Baskin Robbins|    5|
|  Texas Roadhouse|    5|
|           Subway|    5|
|   Dunkin' Donuts|    5|
|      Burger King|    5|
|  Cafe Coffee Day|    5|
|            Giani|    4|
|        Keventers|    4|
+-----------------+-----+
only showing top 20 rows

